In [1]:
# project
from src.data_preprocessing import data_text_management, text_management, tokenizer_transform
from src.utils import performance

# libraries
import pandas as pd
import numpy as np
import json
import warnings

# model libraries
import tensorflow as tf
import keras
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.text import tokenizer_from_json
from tensorflow.keras.preprocessing import sequence

warnings.filterwarnings('ignore')

c:\Users\daalvarado\CSvenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# creamos el objeto
dtm = data_text_management()

In [3]:
# llamamos al modelo
ruta_modelo = r'./model/my_model.keras'
#ruta_modelo = r'model/my_model.keras'
model = load_model(ruta_modelo)

AttributeError: module 'ml_dtypes' has no attribute 'float8_e3m4'


In [4]:
prueba = [
    "El café en este lugar es increíblemente aromático y el ambiente es perfecto para relajarse. ¡Muy recomendado!",
    "El servicio fue lento y las porciones pequeñas para el precio. No lo recomendaría.",
    "El libro tiene una trama cautivadora y personajes memorables. Me mantuvo enganchado de principio a fin.",
    "La película fue predecible y los efectos especiales parecían anticuados. Me decepcionó.",
    "Este champú dejó mi cabello suave y brillante, y el aroma es maravilloso. Definitivamente lo volveré a comprar.",
    "La aplicación se traba constantemente y consume mucha batería. No vale la pena descargarla.",
    "La chaqueta es súper cómoda y tiene un diseño moderno. Perfecta para el invierno.",
    "La comida llegó fría y con un sabor mediocre. No volveré a pedir de aquí.",
    "El curso es muy claro y los ejemplos son prácticos. Aprendí muchísimo en poco tiempo.",
    "Las instrucciones no eran claras y el producto vino defectuoso. Un desperdicio de dinero."
]
sentimiento = [
    'positivo',
    'negativo',
    'positivo',
    'negativo',
    'positivo',
    'negativo',
    'positivo',
    'negativo',
    'positivo',
    'negativo'
]
resenias = pd.DataFrame({'review_es': prueba, 'sentimiento': sentimiento})
resenias = resenias.sample(frac=1)

In [5]:
# limpiando el texto de la reseña
resenias = dtm.cleaning_pipeline(resenias, 'review_es', 'review_es_mod')

In [6]:
# cambiamos las etiquetas a número entero
resenias['sentimiento_token'] = resenias['sentimiento'].map({'positivo' : 1, 'negativo' : 0})

In [7]:
# cargamos el tokenizador
with open(r'./model/cs_tokenizer.json') as token:
    data = json.load(token)
    tokenizer = tokenizer_from_json(data)

In [8]:
# cargamos el maxlen
with open(r'./model/maxlen', 'r') as ml:
    maxlen = int(ml.read())

In [9]:
# tokenizamos
review_text_resenias = tokenizer.texts_to_sequences(resenias['review_es_mod'])

In [10]:
# ajustamos el tamaño del array
review_text_resenias = sequence.pad_sequences(review_text_resenias, maxlen=maxlen, padding='post')

In [11]:
# realizamos la predicción
prediccion = model.predict(review_text_resenias)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 342ms/step


In [12]:
# vemos el array de la predicción y tomamos el valor más grande
prediccion

array([[0.4060521 , 0.6255288 ],
       [0.83040816, 0.33909693],
       [0.7358229 , 0.42251828],
       [0.2581272 , 0.73658586],
       [0.16971155, 0.8247856 ],
       [0.2281016 , 0.77892953],
       [0.5757233 , 0.51500374],
       [0.8021835 , 0.35772872],
       [0.1413466 , 0.855077  ],
       [0.8556688 , 0.32632038]], dtype=float32)

In [13]:
# extraemos el índice de la celda con mayor número por fila, este es el resultado de la prediccion
prediccion_clase = prediccion.argmax(axis=-1)

In [14]:
# lo agregamos al dataframe
resenias['predicted'] = prediccion_clase

In [15]:
# mapeamos de los tokens a positivo/negativo
resenias['predicted_text'] = resenias['predicted'].map({1 : 'positivo', 0 : 'negativo'})

#### probando con un conjunto de datos más grande

Dataset: ./data/netflix/film_reviews_result.csv

In [16]:
# importamos el dataset mayor
netflix = pd.read_csv(r'./data/netflix/film_reviews_result.csv', sep='|')

In [17]:
# extraemos las columnas útiles
netflixuseful = netflix[['film_name', 'review_text', 'review_rate']]

In [18]:
# limpiamos y lemmatizamos el texto
dtm.cleaning_pipeline(netflixuseful, 'review_text', 'review_text_mod')

,film_name,review_text,review_rate,review_text_mod
0,Respira (Serie de TV),"Como fan de las series españolas y de Najwa, e...",1,fan serie español Najwa doler serie malo Manu ...
1,Respira (Serie de TV),Todo lo malo que puede tener una serie lo pose...,2,malo poder tener serie poseer Respira diálogo ...
2,Respira (Serie de TV),"La serie es un drama médico que intenta ""copia...",1,serie drama médico intentar copiar estilema ya...
3,Respira (Serie de TV),Nadie te obliga a ver nada que no quieras ver ...,1,nadie obligar ver querer ver si dinero público...
4,Respira (Serie de TV),Está serie da vergüenza ajena. Una serie donde...,1,serie dar vergüenza ajenir serie parecer único...
...,...,...,...,...
10053,"Confiesa, Fletch",Un misterioso asesinato provoca diversión y de...,6,misterioso asesinato provocar diversión descon...
10054,"Confiesa, Fletch","Empieza bien, pero va perdiendo fuerza y coher...",4,empezar bien ir perder fuerza coherencia medid...
10055,Pesadillas y ensoñaciones: El huérfano (TV),"Segunda entrega de la serie ""Pesadillas y enso...",4,segundo entrega serie Pesadillas ensoñación pa...
10056,Pesadillas y ensoñaciones: La vieja casa (TV),Con Old House comienza la serie de siete episo...,5,Old House comenzar serie siete episodio Pesadi...


In [19]:
# creamos una función que transforme una nota de entre 0-10 a binario
def rating(df : pd.DataFrame, col_in : str | int, col_out : str | int) -> pd.DataFrame:
    conditions = [
        (df[col_in] >= 0) & (df[col_in] <= 5),
        (df[col_in] > 5) & (df[col_in] <= 10)]
    labels = [0, 1]
    df[col_out] = np.select(conditions, labels, default=np.nan)
    return df

In [20]:
# aplicamos la función
netflixuseful = rating(netflixuseful, 'review_rate', 'review_rate_binary')

In [21]:
# cambiamos el dato de float a entero
netflixuseful['review_rate_binary'] = netflixuseful['review_rate_binary'].astype('Int64')

In [22]:
# cargamos el tokenizador
with open(r'./model/cs_tokenizer.json') as token:
    data = json.load(token)
    tokenizer = tokenizer_from_json(data)

# cargamos el maxlen
with open(r'./model/maxlen', 'r') as ml:
    maxlen = int(ml.read())

In [23]:
# tokenizamos
review_text_mod = tokenizer.texts_to_sequences(netflixuseful['review_text_mod'])

# ajustamos el tamaño del array
review_text_mod = sequence.pad_sequences(review_text_mod, maxlen=maxlen, padding='post')

In [24]:
# realizamos la predicción
prediccion = model.predict(review_text_mod)

315/315 ━━━━━━━━━━━━━━━━━━━━ 41s 131ms/step


In [25]:
# vemos el array de la predicción y tomamos el valor más grande
prediccion

array([[0.9759574 , 0.16008608],
       [0.9102827 , 0.27514493],
       [0.6606826 , 0.50130135],
       ...,
       [0.9563676 , 0.21101898],
       [0.50294805, 0.62683856],
       [0.57258785, 0.6007124 ]], dtype=float32)

In [26]:
# extraemos el índice de la celda con mayor número por fila, este es el resultado de la prediccion
prediccion_clase = prediccion.argmax(axis=-1)

In [27]:
# lo agregamos al dataframe
netflixuseful['predicted'] = prediccion_clase

In [28]:
# mapeamos de los tokens a positivo/negativo
netflixuseful['predicted_text'] = netflixuseful['predicted'].map({1 : 'positivo', 0 : 'negativo'})

In [30]:
netflixuseful['Accuracy'] = netflixuseful['review_rate_binary'] == netflixuseful['predicted']

In [41]:
# porcentaje aciertos
accuracy = netflixuseful.groupby(by='Accuracy').size()
accuracy_per = accuracy[1] / (accuracy[0] + accuracy[1])
accuracy_per*100

77.32153509644064